**Tutorial 2:**

We are going to use LangChain to chain LLMs together: make the output of a model become the input of another

Disclaimer: We use the open-source HuggingFace model here to generate output. Better/premium models may produce better output

Step 1: Setup (Refer to Tutorial 1)

In [1]:
# Keys here: https://huggingface.co/docs/hub/security-tokens

In [2]:
f = open("./hfhub_key")
hfhub_key = f.read().strip()
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hfhub_key
from langchain import HuggingFaceHub
from langchain import PromptTemplate, LLMChain

In [3]:
repo_id = "google/flan-t5-xxl"
llm = HuggingFaceHub(
   repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 500})

Step 3: Let's define two LLMChains. One that generates an essay for a topic. Another that generates feedback for an essay.

In [4]:
essay = """You are a computer science student. Given the topic of the essay, it is your job to write a short essay.

Topic: {topic}
Essay: This is the text for the above essay:"""
essay_prompt_template = PromptTemplate(input_variables=["topic"], template=essay)

In [5]:
essay_chain = LLMChain(llm=llm, prompt=essay_prompt_template)

In [13]:
feedback = """You are a course instructor from the University. Given the text of an essay, it is your job to say if the essay is correct.

Essay Text:
{essay_text}
Feedback from a University course instructor of the above assignment:"""
feedback_prompt_template = PromptTemplate(input_variables=["essay_text"], template=feedback)

In [14]:
feedback_chain = LLMChain(llm=llm, prompt=feedback_prompt_template)

Step 4: Let's chain them together

In [15]:
from langchain.chains import SimpleSequentialChain

In [16]:
overall_chain = SimpleSequentialChain(chains=[essay_chain, feedback_chain], verbose=True)

In [17]:
feedback = overall_chain.run("linked lists")



> Entering new SimpleSequentialChain chain...
Linked lists are collections of pointers to elements of a collection.
Yes

> Finished chain.


In [18]:
feedback

'Yes'

Note:
    - OpenAI API allows chain-of-thought to be elucidated explicitly in the prompt. The mechanism above can be useful when using (free) open-source models